<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [7]</a>'.</span>

# Prepare data for Pipeline

## Setup and settings

In [1]:
# Importing packages
import os
import yaml
import logging
import pandas as pd
from Bio.Seq import Seq
from qiime2 import Artifact, Visualization, Metadata
from qiime2.plugins import demux
from qiime2.plugins.cutadapt.methods import trim_paired, trim_single
from qiime2.plugins.demux.methods import filter_samples

### Receiving the parameters

The following cell can receive parameters using the [papermill](https://papermill.readthedocs.io/en/latest/) tool.

In [2]:
params_path = os.path.join('..', 'params', 'parameter-set-01.yaml')
experiment_name = 'exp-01'
base_dir = os.path.join('/', 'home', 'username', 'pipeline-dir')
manifest_file = os.path.join(base_dir, 'data', 'raw', 'manifest', 'manifest.csv')
replace_files = False
trim = None
metadata_file = None
threads = 1

In [3]:
# Parameters
experiment_name = "ana-flavia-all"
base_dir = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri"
manifest_file = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/manifest-all-ana.csv"
metadata_file = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/metadata-all-ana-types.tsv"
class_col = "group-id"
classifier_file = "/home/lauro/nupeb/rede-micro/models/silva-138-99-nb-classifier.qza"
top_n = 20
replace_files = False
phred = 20
trunc_f = 0
trunc_r = 0
overlap = 12
threads = 6
trim = {
    "overlap": 8,
    "forward_primer": "CCTACGGGRSGCAGCAG",
    "reverse_primer": "GGACTACHVGGGTWTCTAAT",
}


### Defining names and paths

In [4]:
# Define the output folder path
out_dir = os.path.join(base_dir, 'experiments', experiment_name, 'qiime-artifacts')
img_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name, 'imgs'))

# Create path if it not exists
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)
    print(f'New qiime-artifacts folder path created: {out_dir}')

# Create path if it not exists
if not os.path.isdir(img_folder):
    os.makedirs(img_folder)
    print(f'New img folder path created: {img_folder}')

# Define the output artifact full path
demux_file = os.path.join(out_dir, 'demux-paired.qza')
demux_view = os.path.join(out_dir, 'demux-paired.qzv')
demux_file_trim = os.path.join(out_dir, 'demux-paired-trim.qza')
demux_view_trim = os.path.join(out_dir, 'demux-paired-trim.qzv')

## Step execution

This Step import all `fastq` files in a **QIIME2 Artifact** object and save it to a new `qza` file of `SampleData[PairedEndSequencesWithQuality]`

In [5]:
manifest_df = pd.read_csv(manifest_file)
n_directions = len(manifest_df['direction'].unique())
if n_directions == 1:
    d_type = 'SampleData[SequencesWithQuality]'
    v_type = 'SingleEndFastqManifestPhred33'
elif n_directions == 2:
    d_type = 'SampleData[PairedEndSequencesWithQuality]'
    v_type = 'PairedEndFastqManifestPhred33'
else:
    print(f'ERROR: invalid number of directions {n_directions}')

In [6]:
# If the metadata file is not defined, use the default metadata file
metadata_qa = Metadata.load(metadata_file)
metadata_qa.to_dataframe()

,sample-desc,group-id,group-desc
sample-id,,,
S210421121673,N9.01,NC,Ninhada-controle
S210421121674,N11.01,NC,Ninhada-controle
S210421121675,N9.05,NC,Ninhada-controle
S210421121676,N11.02,NC,Ninhada-controle
S210421121677,N9.03,NR,Ninhada-reduzida
S210421121678,N10.03,NR,Ninhada-reduzida
S210421121679,N10.04,NR,Ninhada-reduzida
S210421121680,N10.07,NR,Ninhada-reduzida
S210421121681,N10.08,NR,Ninhada-reduzida


In [7]:
print(manifest_file)
!head {manifest_file}

/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/manifest-all-ana.csv
sample-id,absolute-filepath,direction
S210421121673,/home/lauro/nupeb/dados_brutos_rede_genoma/data/16s_karina/workspace/decompressed_reads/210421121673-1-1-1_S81_L001_R1_001.fastq,forward
S210421121673,/home/lauro/nupeb/dados_brutos_rede_genoma/data/16s_karina/workspace/decompressed_reads/210421121673-1-1-1_S81_L001_R2_001.fastq,reverse
S210421121674,/home/lauro/nupeb/dados_brutos_rede_genoma/data/16s_karina/workspace/decompressed_reads/210421121674-1-1-1_S83_L001_R1_001.fastq,forward
S210421121674,/home/lauro/nupeb/dados_brutos_rede_genoma/data/16s_karina/workspace/decompressed_reads/210421121674-1-1-1_S83_L001_R2_001.fastq,reverse
S210421121675,/home/lauro/nupeb/dados_brutos_rede_genoma/data/16s_karina/workspace/decompressed_reads/210421121675-1-1-1_S85_L001_R1_001.fastq,forward
S210421121675,/home/lauro/nupeb/dados_brutos_rede_genoma/data/16s_karina/workspace/decompressed_reads/210421121675-1-1-1_

In [8]:
# Import data and create an artifact object
artifact = Artifact.import_data(
    d_type, 
    manifest_file, 
    view_type=v_type)

#     artifact = filter_samples(demux=artifact, metadata=metadata_qa).filtered_demux
# Save the artifact object to a new qza file
artifact.save(demux_file)

'/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/experiments/ana-flavia-all/qiime-artifacts/demux-paired.qza'

In [ ]:
# If the demux file does not exist or if the old file will be replaced
if not os.path.isfile(demux_file) or replace_files:
    
    # Import data and create an artifact object
    artifact = Artifact.import_data(
        d_type, 
        manifest_file, 
        view_type=v_type)

#     artifact = filter_samples(demux=artifact, metadata=metadata_qa).filtered_demux
    # Save the artifact object to a new qza file
    artifact.save(demux_file)

else:
    artifact = Artifact.load(demux_file)

In [9]:
if not os.path.isfile(demux_view) or replace_files: 
    # Generate e visualization of the Artifact
    demux_summary = demux.visualizers.summarize(artifact)

    # Save a new visualization file based on the qza file
    Visualization.save(demux_summary.visualization, filepath=demux_view)
    
    demux_view_obj = demux_summary.visualization
else:
    demux_view_obj = Visualization.load(demux_view)

In [10]:
demux_file

'/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/experiments/ana-flavia-all/qiime-artifacts/demux-paired.qza'

## Step report


In [ ]:
print(demux_view_obj)

In [ ]:
# Render Visualization
demux_view_obj

[cutadapt](https://docs.qiime2.org/2022.2/plugins/available/cutadapt/trim-paired/)

We trim the forward primer and the reverse complement of the reverse primer from the forward reads. We trim the reverse primer and reverse complement of the forward primer from the reverse reads.

In [13]:
demux_file_trim = os.path.join(out_dir, 'demux-paired-trim.qza')
demux_view_trim = os.path.join(out_dir, 'demux-paired-trim.qzv')

if trim and (not os.path.isfile(demux_file_trim) or replace_files):
    forward_primer = [trim['forward_primer']] # ['CCTACGGGRSGCAGCAG']
    reverse_primer = [trim['reverse_primer']] # ['GGACTACHVGGGTWTCTAAT']
    forward_reverse_complement = [str(Seq(forward_primer[0]).reverse_complement())]
    reverse_reverse_complement = [str(Seq(reverse_primer[0]).reverse_complement())]

    res = None
    
    if n_directions == 1:
        res = trim_single(
            demultiplexed_sequences=artifact,
            front=forward_primer,
            adapter=reverse_reverse_complement,
            cores=threads,
            indels=False,
            match_read_wildcards=True,
            match_adapter_wildcards=True,
            error_rate=0.01,
            discard_untrimmed=True,
        ).trimmed_sequences
    elif n_directions == 2:
        res = trim_paired(
            demultiplexed_sequences=artifact,
            front_f=forward_primer,
            front_r=reverse_primer,
            # adapter_f=reverse_reverse_complement,
            # adapter_r=forward_reverse_complement,
            cores=threads,
            overlap=trim['overlap'],
            indels=False,
            match_read_wildcards=True,
            match_adapter_wildcards=True,
            error_rate=0.01,
            discard_untrimmed=True,
        ).trimmed_sequences
    else:
        print(f'ERROR: invalid number of directions {n_directions}')

    if res is not None:
        res.save(demux_file_trim)
        res_view = demux.visualizers.summarize(res).visualization
        Visualization.save(res_view, filepath=demux_view_trim)

Running external command line application. This may print messages to stdout and/or stderr.
The commands to be run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt --cores 6 --error-rate 0.01 --times 1 --overlap 8 --minimum-length 1 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-oyazuqe_/S210421121673_0_L001_R1_001.fastq.gz -p /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-oyazuqe_/S210421121673_1_L001_R2_001.fastq.gz --front CCTACGGGRSGCAGCAG -G GGACTACHVGGGTWTCTAAT --no-indels --match-read-wildcards --discard-untrimmed /tmp/qiime2-archive-7lm7gvl9/068f0c34-15b0-437a-a67e-bb0223aa7625/data/S210421121673_0_L001_R1_001.fastq.gz /tmp/qiime2-archive-7lm7gvl9/068f0c34-15b0-437a-a67e-bb0223aa7625/data/S210421121673_1_L001_R2_001.fastq.gz

This is cutadapt 3.5 with Python 3.8.12
Command line parameters: --cores 6 --error-rate 0.01 --times 1 --overlap 8 --minimum-length 1 -o /tmp/q2-CasavaOneEightSingleLan

Finished in 6.49 s (9 µs/read; 6.59 M reads/minute).

=== Summary ===

Total read pairs processed:            712,750
  Read 1 with adapter:                 694,205 (97.4%)
  Read 2 with adapter:                 664,033 (93.2%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           65,715 (9.2%)
Pairs written (passing filters):       647,035 (90.8%)

Total basepairs processed:   363,502,500 bp
  Read 1:   217,388,750 bp
  Read 2:   146,113,750 bp
Total written (filtered):    301,650,677 bp (83.0%)
  Read 1:   184,115,305 bp
  Read 2:   117,535,372 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 694205 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	4	10.9	0	4
9	9	2.7	0	9
10	5	0.7	0	5
11	14	0.2	0	14
12	8	0.0	0	8
13	6	0.0	0	6
14	30	0.0	0	30
15	18	0.0	0	18
16	33	0.0	0	33
17	183	0.0	0	1

Finished in 2.95 s (9 µs/read; 6.44 M reads/minute).

=== Summary ===

Total read pairs processed:            316,849
  Read 1 with adapter:                 308,351 (97.3%)
  Read 2 with adapter:                 292,785 (92.4%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           31,765 (10.0%)
Pairs written (passing filters):       285,084 (90.0%)

Total basepairs processed:   161,592,990 bp
  Read 1:    96,638,945 bp
  Read 2:    64,954,045 bp
Total written (filtered):    132,905,965 bp (82.2%)
  Read 1:    81,120,487 bp
  Read 2:    51,785,478 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 308351 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	4	4.8	0	4
9	1	1.2	0	1
10	1	0.3	0	1
11	9	0.1	0	9
12	4	0.0	0	4
13	5	0.0	0	5
14	17	0.0	0	17
15	20	0.0	0	20
16	21	0.0	0	21
17	83	0.0	0	83
1

Finished in 1.71 s (10 µs/read; 5.82 M reads/minute).

=== Summary ===

Total read pairs processed:            165,821
  Read 1 with adapter:                 161,411 (97.3%)
  Read 2 with adapter:                 154,014 (92.9%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           15,834 (9.5%)
Pairs written (passing filters):       149,987 (90.5%)

Total basepairs processed:    84,568,710 bp
  Read 1:    50,575,405 bp
  Read 2:    33,993,305 bp
Total written (filtered):     69,924,570 bp (82.7%)
  Read 1:    42,679,301 bp
  Read 2:    27,245,269 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 161411 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	3	2.5	0	3
9	1	0.6	0	1
10	3	0.2	0	3
11	4	0.0	0	4
12	1	0.0	0	1
13	3	0.0	0	3
14	8	0.0	0	8
15	1	0.0	0	1
16	4	0.0	0	4
17	46	0.0	0	46
18	2512

Finished in 2.48 s (10 µs/read; 6.17 M reads/minute).

=== Summary ===

Total read pairs processed:            254,393
  Read 1 with adapter:                 247,382 (97.2%)
  Read 2 with adapter:                 234,645 (92.2%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           26,087 (10.3%)
Pairs written (passing filters):       228,306 (89.7%)

Total basepairs processed:   129,740,430 bp
  Read 1:    77,589,865 bp
  Read 2:    52,150,565 bp
Total written (filtered):    106,436,280 bp (82.0%)
  Read 1:    64,964,669 bp
  Read 2:    41,471,611 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 247382 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	3	3.9	0	3
9	3	1.0	0	3
10	2	0.2	0	2
11	1	0.1	0	1
12	6	0.0	0	6
13	5	0.0	0	5
14	13	0.0	0	13
15	8	0.0	0	8
16	5	0.0	0	5
17	82	0.0	0	82
18	3

...

Finished in 1.66 s (11 µs/read; 5.45 M reads/minute).

=== Summary ===

Total read pairs processed:            151,058
  Read 1 with adapter:                 146,859 (97.2%)
  Read 2 with adapter:                 139,367 (92.3%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           15,510 (10.3%)
Pairs written (passing filters):       135,548 (89.7%)

Total basepairs processed:    77,039,580 bp
  Read 1:    46,072,690 bp
  Read 2:    30,966,890 bp
Total written (filtered):     63,193,980 bp (82.0%)
  Read 1:    38,571,401 bp
  Read 2:    24,622,579 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 146859 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
9	3	0.6	0	3
10	2	0.1	0	2
11	3	0.0	0	3
12	1	0.0	0	1
13	3	0.0	0	3
14	5	0.0	0	5
15	5	0.0	0	5
16	3	0.0	0	3
17	39	0.0	0	39
18	22910	0.0	0	229

Finished in 2.36 s (10 µs/read; 6.19 M reads/minute).

=== Summary ===

Total read pairs processed:            243,179
  Read 1 with adapter:                 236,854 (97.4%)
  Read 2 with adapter:                 223,595 (91.9%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           25,304 (10.4%)
Pairs written (passing filters):       217,875 (89.6%)

Total basepairs processed:   124,021,290 bp
  Read 1:    74,169,595 bp
  Read 2:    49,851,695 bp
Total written (filtered):    101,573,655 bp (81.9%)
  Read 1:    61,995,458 bp
  Read 2:    39,578,197 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 236854 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	2	3.7	0	2
9	2	0.9	0	2
10	1	0.2	0	1
11	3	0.1	0	3
12	1	0.0	0	1
13	3	0.0	0	3
14	14	0.0	0	14
15	12	0.0	0	12
16	9	0.0	0	9
17	66	0.0	0	66
18

Finished in 0.94 s (12 µs/read; 5.16 M reads/minute).

=== Summary ===

Total read pairs processed:             80,755
  Read 1 with adapter:                  78,601 (97.3%)
  Read 2 with adapter:                  74,038 (91.7%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:            8,658 (10.7%)
Pairs written (passing filters):        72,097 (89.3%)

Total basepairs processed:    41,185,050 bp
  Read 1:    24,630,275 bp
  Read 2:    16,554,775 bp
Total written (filtered):     33,611,590 bp (81.6%)
  Read 1:    20,515,535 bp
  Read 2:    13,096,055 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 78601 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
10	1	0.1	0	1
11	1	0.0	0	1
12	1	0.0	0	1
13	2	0.0	0	2
14	5	0.0	0	5
15	1	0.0	0	1
16	2	0.0	0	2
17	22	0.0	0	22
18	12274	0.0	0	12274
19	14235	0

Finished in 1.48 s (11 µs/read; 5.65 M reads/minute).

=== Summary ===

Total read pairs processed:            139,176
  Read 1 with adapter:                 135,203 (97.1%)
  Read 2 with adapter:                 127,496 (91.6%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           15,223 (10.9%)
Pairs written (passing filters):       123,953 (89.1%)

Total basepairs processed:    70,979,760 bp
  Read 1:    42,448,680 bp
  Read 2:    28,531,080 bp
Total written (filtered):     57,785,990 bp (81.4%)
  Read 1:    35,270,977 bp
  Read 2:    22,515,013 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 135203 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
9	1	0.5	0	1
11	3	0.0	0	3
12	2	0.0	0	2
14	4	0.0	0	4
15	3	0.0	0	3
16	4	0.0	0	4
17	35	0.0	0	35
18	21044	0.0	0	21044
19	24431	0.0	0	24431
20

Finished in 3.10 s (10 µs/read; 6.09 M reads/minute).

=== Summary ===

Total read pairs processed:            314,325
  Read 1 with adapter:                 305,481 (97.2%)
  Read 2 with adapter:                 290,205 (92.3%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           32,137 (10.2%)
Pairs written (passing filters):       282,188 (89.8%)

Total basepairs processed:   160,305,750 bp
  Read 1:    95,869,125 bp
  Read 2:    64,436,625 bp
Total written (filtered):    131,558,190 bp (82.1%)
  Read 1:    80,299,014 bp
  Read 2:    51,259,176 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 305481 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	3	4.8	0	3
9	5	1.2	0	5
10	6	0.3	0	6
11	5	0.1	0	5
12	6	0.0	0	6
13	1	0.0	0	1
14	13	0.0	0	13
15	5	0.0	0	5
16	19	0.0	0	19
17	86	0.0	0	86
18

In [14]:
res_view

<visualization: Visualization uuid: 19260d63-9a57-45b5-a582-8552cc8a0e4d>

In [ ]:
demux_path = os.path.join('..', 'qiime-artifacts', 'demux-paired-trim.qza')
!ls {demux_path}

In [ ]:
filtered_demux_path = os.path.join(out_dir, 'demux-paired-trim-filtered.qza')

In [ ]:
demux_path

In [ ]:
metadata_qa = Metadata.load(metadata_file)
metadata_qa

In [ ]:
metadata_f = os.path.join('..', '..', '..', 'data', 'raw', 'metadata-all-ana.tsv')
!ls {metadata_f}
metadata_art = Metadata.load(metadata_f)
metadata_art

aux_art = Artifact.load(demux_path)
aux_art

In [ ]:
filtered_demux_path = '../qiime-artifacts/demux-paired-trim-filtered.qza'

In [ ]:
%%bash
metadata_f="../../../data/raw/metadata-all-ana.tsv"
demux_path="../qiime-artifacts/demux-paired-trim.qza"
filtered_demux_path="../qiime-artifacts/demux-paired-trim-filtered.qza"

qiime demux filter-samples --i-demux ${demux_path} --m-metadata-file ${metadata_f} --o-filtered-demux ${filtered_demux_path}

In [ ]:
from qiime2.plugins.demux.methods import filter_samples

res2 = filter_samples(demux=aux_art, metadata=metadata_art).filtered_demux